In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from collections import defaultdict

from surprise.model_selection import KFold

from surprise.model_selection.validation  import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans

import os
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

from surprise.model_selection import train_test_split

from sklearn.metrics import  roc_curve
from sklearn.metrics import roc_auc_score
from collections import defaultdict

from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.matrix_factorization import SVD

import seaborn as sns

import csv
import re


In [83]:
bike_data = pd.read_csv('../input/project4data/day.csv')

bike_data

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [84]:
video_data = pd.read_csv('../input/project4data/transcoding_mesurment.tsv', sep='\t')
video_data

,id,duration,codec,width,height,bitrate,framerate,i,p,b,...,p_size,b_size,size,o_codec,o_bitrate,o_framerate,o_width,o_height,umem,utime
0,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,176,144,22508,0.612
1,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,320,240,25164,0.980
2,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,480,360,29228,1.216
3,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,640,480,34316,1.692
4,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,1280,720,58528,3.456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68779,ZWEN-71BqPs,972.27100,h264,480,360,278822,29.000000,560,28580,0,...,26561730,0,33886358,flv,242000,24.00,640,480,88692,1.552
68780,zWQN-bqqg0o,129.88100,vp8,640,480,639331,30.162790,36,3855,0,...,9503846,0,10379630,mpeg4,539000,29.97,1920,1080,107524,18.557
68781,zX17-vi0sqQ,249.68000,vp8,320,240,359345,25.068274,129,6113,0,...,9456514,0,11215178,flv,539000,12.00,176,144,88708,0.752
68782,zyiT-TzxIpk,183.62334,h264,1280,720,2847539,29.000000,98,5405,0,...,60113035,0,65359329,mpeg4,539000,12.00,320,240,88724,5.444


In [87]:

bike_data_df = pd.DataFrame(data=bike_data)
video_data_df = pd.DataFrame(data=video_data)


In [89]:
video_data_df.dtypes

id              object
duration       float64
codec           object
width            int64
height           int64
bitrate          int64
framerate      float64
i                int64
p                int64
b                int64
frames           int64
i_size           int64
p_size           int64
b_size           int64
size             int64
o_codec         object
o_bitrate        int64
o_framerate    float64
o_width          int64
o_height         int64
umem             int64
utime          float64
dtype: object

In [91]:
bike_data_df.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [92]:
# one hot encoding - bike_data
from sklearn.preprocessing import OneHotEncoder 


#http://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example

#one hot encode weathersit for sure, season probably, weekday (0-6) maybe
# label encode the date into days of the month
encoded_bike_data = bike_data_df.copy()

encoded_bike_data['dteday'] = pd.to_datetime(bike_data_df['dteday']).dt.strftime('%d')
encoded_bike_data['dteday'] = pd.to_numeric(encoded_bike_data['dteday'])

# use pd.concat to join the new columns with your original dataframe
df_season = pd.concat([encoded_bike_data,pd.get_dummies(encoded_bike_data['season'], prefix='season')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_season.drop(['season'],axis=1, inplace=True)

df_weathersit = pd.concat([df_season, pd.get_dummies(df_season['weathersit'], prefix = 'weathersit')], axis=1)
df_weathersit.drop(['weathersit'], axis=1, inplace=True)


df_holiday = pd.concat([df_weathersit, pd.get_dummies(df_weathersit['holiday'], prefix = 'holiday')], axis=1)
df_holiday.drop(['holiday'], axis=1, inplace=True)

df_workday = pd.concat([df_holiday, pd.get_dummies(df_holiday['workingday'], prefix = 'workingday')], axis=1)
df_workday.drop(['workingday'], axis=1, inplace=True)
bike_data_proc_df = df_workday
print(bike_data_proc_df.columns)
bike_data_proc_df


Index(['instant', 'dteday', 'yr', 'mnth', 'weekday', 'temp', 'atemp', 'hum',
       'windspeed', 'casual', 'registered', 'cnt', 'season_1', 'season_2',
       'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3',
       'holiday_0', 'holiday_1', 'workingday_0', 'workingday_1'],
      dtype='object')


,instant,dteday,yr,mnth,weekday,temp,atemp,hum,windspeed,casual,...,season_2,season_3,season_4,weathersit_1,weathersit_2,weathersit_3,holiday_0,holiday_1,workingday_0,workingday_1
0,1,1,0,1,6,0.344167,0.363625,0.805833,0.160446,331,...,0,0,0,0,1,0,1,0,1,0
1,2,2,0,1,0,0.363478,0.353739,0.696087,0.248539,131,...,0,0,0,0,1,0,1,0,1,0
2,3,3,0,1,1,0.196364,0.189405,0.437273,0.248309,120,...,0,0,0,1,0,0,1,0,0,1
3,4,4,0,1,2,0.200000,0.212122,0.590435,0.160296,108,...,0,0,0,1,0,0,1,0,0,1
4,5,5,0,1,3,0.226957,0.229270,0.436957,0.186900,82,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,27,1,12,4,0.254167,0.226642,0.652917,0.350133,247,...,0,0,0,0,1,0,1,0,0,1
727,728,28,1,12,5,0.253333,0.255046,0.590000,0.155471,644,...,0,0,0,0,1,0,1,0,0,1
728,729,29,1,12,6,0.253333,0.242400,0.752917,0.124383,159,...,0,0,0,0,1,0,1,0,1,0
729,730,30,1,12,0,0.255833,0.231700,0.483333,0.350754,364,...,0,0,0,1,0,0,1,0,1,0


In [93]:
print(video_data_df.columns)
print(video_data)

Index(['id', 'duration', 'codec', 'width', 'height', 'bitrate', 'framerate',
       'i', 'p', 'b', 'frames', 'i_size', 'p_size', 'b_size', 'size',
       'o_codec', 'o_bitrate', 'o_framerate', 'o_width', 'o_height', 'umem',
       'utime'],
      dtype='object')
                id   duration  codec  width  height  bitrate  framerate    i  \
0      04t6-jw9czg  130.35667  mpeg4    176     144    54590  12.000000   27   
1      04t6-jw9czg  130.35667  mpeg4    176     144    54590  12.000000   27   
2      04t6-jw9czg  130.35667  mpeg4    176     144    54590  12.000000   27   
3      04t6-jw9czg  130.35667  mpeg4    176     144    54590  12.000000   27   
4      04t6-jw9czg  130.35667  mpeg4    176     144    54590  12.000000   27   
...            ...        ...    ...    ...     ...      ...        ...  ...   
68779  ZWEN-71BqPs  972.27100   h264    480     360   278822  29.000000  560   
68780  zWQN-bqqg0o  129.88100    vp8    640     480   639331  30.162790   36   
68781  zX17-vi0sq

In [94]:
# one hot encoding - video data
video_data_noid = video_data_df.copy()
video_data_noid.drop(['id'], axis=1, inplace=True)


df_codec = pd.concat([video_data_noid, pd.get_dummies(video_data_noid['codec'], prefix = 'codec')], axis=1)
df_codec.drop(['codec'], axis=1, inplace=True)


df_o_codec = pd.concat([df_codec, pd.get_dummies(df_codec['o_codec'], prefix = 'o_codec')], axis=1)
df_o_codec.drop(['o_codec'], axis=1, inplace=True)

video_data_proc_df = df_o_codec.copy()
video_data_proc_df

,duration,width,height,bitrate,framerate,i,p,b,frames,i_size,...,umem,utime,codec_flv,codec_h264,codec_mpeg4,codec_vp8,o_codec_flv,o_codec_h264,o_codec_mpeg4,o_codec_vp8
0,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,22508,0.612,0,0,1,0,0,0,1,0
1,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,25164,0.980,0,0,1,0,0,0,1,0
2,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,29228,1.216,0,0,1,0,0,0,1,0
3,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,34316,1.692,0,0,1,0,0,0,1,0
4,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,58528,3.456,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68779,972.27100,480,360,278822,29.000000,560,28580,0,29140,7324628,...,88692,1.552,0,1,0,0,1,0,0,0
68780,129.88100,640,480,639331,30.162790,36,3855,0,3891,875784,...,107524,18.557,0,0,0,1,0,0,1,0
68781,249.68000,320,240,359345,25.068274,129,6113,0,6242,1758664,...,88708,0.752,0,0,0,1,1,0,0,0
68782,183.62334,1280,720,2847539,29.000000,98,5405,0,5503,5246294,...,88724,5.444,0,1,0,0,0,0,1,0


In [97]:
#question 7 - bike data
from sklearn.preprocessing import StandardScaler

#standardize all feature columns

col_names = bike_data_proc_df.columns
features = bike_data_proc_df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

#assign scaled results to the dataset copy
bike_data_proc_df[col_names] = features
print(bike_data_proc_df)

print(np.mean(bike_data_proc_df['atemp']))


      instant    dteday        yr      mnth   weekday      temp     atemp  \
0   -1.729683 -1.674108 -1.001369 -1.600161  1.498809 -0.826662 -0.679946   
1   -1.724944 -1.560522 -1.001369 -1.600161 -1.496077 -0.721095 -0.740652   
2   -1.720205 -1.446936 -1.001369 -1.600161 -0.996930 -1.634657 -1.749767   
3   -1.715466 -1.333351 -1.001369 -1.600161 -0.497782 -1.614780 -1.610270   
4   -1.710728 -1.219765 -1.001369 -1.600161  0.001366 -1.467414 -1.504971   
..        ...       ...       ...       ...       ...       ...       ...   
726  1.710728  1.279122  0.998633  1.588660  0.500513 -1.318665 -1.521108   
727  1.715466  1.392707  0.998633  1.588660  0.999661 -1.323224 -1.346690   
728  1.720205  1.506293  0.998633  1.588660  1.498809 -1.323224 -1.424344   
729  1.724944  1.619879  0.998633  1.588660 -1.496077 -1.309558 -1.490049   
730  1.729683  1.733465  0.998633  1.588660 -0.996930 -1.528225 -1.540482   

          hum  windspeed    casual  ...  season_2  season_3  season_4  \
0 

In [98]:
#question 7 continued

from sklearn.preprocessing import StandardScaler


video_col_names = video_data_proc_df.columns
video_features = video_data_proc_df[video_col_names]
video_scaler = StandardScaler().fit(video_features.values)
video_features = video_scaler.transform(video_features.values)

#assign scaled results to the dataset copy
video_data_proc_df[video_col_names] = video_features
print(video_data_proc_df)

#check that mean of a column is approximately 0
print(np.mean(video_data_proc_df['p']))


       duration     width    height   bitrate  framerate         i         p  \
0     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
1     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
2     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
3     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
4     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
...         ...       ...       ...       ...        ...       ...       ...   
68779  2.387620 -0.312921 -0.218492 -0.378671   0.797072  5.416577  3.628857   
68780 -0.544926  0.032528  0.280232 -0.049625   0.958016 -0.765280 -0.440548   
68781 -0.127879 -0.658369 -0.717217 -0.305176   0.252873  0.331882 -0.068911   
68782 -0.357837  1.414323  1.277682  1.965863   0.797072 -0.033839 -0.185438   
68783  0.028544 -0.969273 -1.116197 -0.582738  -1.555936 -0.470344 -0.503255   

              b    frames    i_size  ..

In [109]:
#question 8 feature selection

#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression

#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression
from sklearn.feature_selection import f_regression, mutual_info_regression


#separate the matrices into features and targets
bike_target = bike_data_proc_df.cnt
video_target = video_data_proc_df.utime

bike_feature_mtx = bike_data_proc_df.drop(['registered','casual','cnt'], axis=1, inplace=False)

#account for dummy variable trap? that is leading to memory leak for video_data with f regression AND mutual info
video_feature_mtx = video_data_proc_df.drop(['o_codec_flv', 'o_codec_h264', 'o_codec_mpeg4','o_codec_vp8','utime'], axis=1, inplace=False)


#f regression better for linear?
print("---- Bike Features ----")
bike_feature_r = f_regression(bike_feature_mtx, bike_target)
print("---- Bike F Regression ----")

print(bike_feature_r)

print("---- Bike Mutual Information ----")
#mutual information better for nonlinear?
bike_feature_mir = mutual_info_regression(bike_feature_mtx, bike_target, copy=True)
print(bike_feature_mir)


---- Bike Features ----
---- Bike F Regression ----
(array([4.76810740e+02, 4.27336147e-01, 3.44890586e+02, 6.20046245e+01,
       3.33109137e+00, 4.73471711e+02, 4.82454311e+02, 7.46194000e+00,
       4.24378416e+01, 3.38467598e+02, 1.59217364e+01, 9.94590876e+01,
       3.14999760e+00, 4.97987371e+01, 2.24590570e+01, 4.45403742e+01,
       3.42144104e+00, 3.42144104e+00, 2.73674228e+00, 2.73674228e+00]), array([1.02076318e-81, 5.13505103e-01, 2.48353990e-63, 1.24311178e-14,
       6.83908070e-02, 2.81062240e-81, 1.85450413e-82, 6.45414333e-03,
       1.35995868e-10, 2.22572389e-62, 7.26872835e-05, 4.79590176e-22,
       7.63443042e-02, 3.97113639e-12, 2.58147987e-06, 4.93394699e-11,
       6.47593579e-02, 6.47593579e-02, 9.84949616e-02, 9.84949616e-02]))
---- Bike Mutual Information ----
[0.90529085 0.         0.27825329 0.37675924 0.04563257 0.3896914
 0.46425812 0.04629649 0.05599795 0.19682043 0.00652323 0.07143707
 0.03356258 0.04469594 0.02004465 0.04359151 0.01147809 0.01147809

In [ ]:
print("---- Video Features ----")

#video_feature_r = f_regression(video_feature_mtx, video_target)
print("---- Video F Regression ----")

print(video_feature_r)

#mutual information better for nonlinear?
#print("---- Video Mutual Information ----")
#video_feature_mir = mutual_info_regression(video_feature_mtx, video_target, copy=True, n_neighbors=5)
#print(video_feature_mir)

#video mutual info still has some memory leak issue

In [ ]:
print(video_data_proc_df.columns)
print(video_feature_r)

In [ ]:
# question 9 theoretical - explain each regression scheme

In [ ]:
#question 10 - choose the best regression scheme for our data and optimal penalty parameter

In [ ]:
#question 11, 12 - more theoretical questions

In [115]:
#train ordinary least squares, lasso, ridge regression separately and compare

from sklearn import linear_model
from sklearn.model_selection  import cross_validate

#full processed sets
#bike_data_proc_df
#video_data_proc_df


#feature sets
#bike_feature_mtx
#video_feature_mtx

#target sets
#bike_target
#video_target

#don't need a split for cross-validation?
#bike_X_train, bike_X_test, bike_y_train, bike_y_test = train_test_split(bike_feature_mtx, bike_target, test_size=0.2, random_state=0)

#reference: https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py

reg = linear_model.LinearRegression()
#reg.fit(x_train, y_train)
#y_pred = regr.predict(x_test)
cv = cross_validate(reg, bike_feature_mtx, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
# y is the target, x is the features
bike_train_MSE = -1* cv['test_score']
bike_test_MSE = -1* cv['train_score']

bike_train_RMSE = np.sqrt(bike_train_MSE)
bike_test_RMSE = np.sqrt(bike_test_MSE)

print("Linear Regression")
print("Bike Training RMSE: " +str(bike_train_RMSE))
print("Bike Testing RMSE: " +str(bike_test_RMSE))


print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))

print("---------------------------------")

reg = linear_model.LinearRegression(fit_intercept=False)
cv = cross_validate(reg, video_feature_mtx, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
# y is the target, x is the features
video_train_MSE = -1* cv['test_score']
video_test_MSE = -1* cv['train_score']

video_train_RMSE = np.sqrt(video_train_MSE)
video_test_RMSE = np.sqrt(video_test_MSE)



print("Training RMSE: " +str(video_train_RMSE))
print("Testing RMSE: " +str(video_test_RMSE))


print("Average Training RMSE: " +str(np.mean(video_train_RMSE)))
print("Average Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))

Bike Training RMSE: [0.27523502 0.34314316 0.52274656 0.37870083 0.30681512 0.34204409
 0.49896039 0.56376921 0.61579663 0.76644172]
Bike Testing RMSE: [0.42793329 0.42131689 0.40611724 0.41822942 0.42475536 0.4239225
 0.40472221 0.39679186 0.39544988 0.38031118]
Average Bike Training RMSE: 0.46136527418917234
Average Bike Testing/Validation RMSE: 0.4099549844623903
---------------------------------
Training RMSE: [0.80313518 0.60576854 0.63036668 0.65296087 0.64839389 0.71570341
 0.58972252 0.62040154 0.5487757  0.83363365]
Testing RMSE: [0.6336046  0.65504959 0.6522016  0.6498069  0.65024913 0.64300449
 0.65717529 0.65353735 0.6611163  0.637935  ]
Average Training RMSE: 0.6648861982307204
Average Testing/Validation RMSE: 0.6493680251620894


In [123]:
# lasso/l1 regression


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

#bike lasso regression
for alphas in alpha_values:
    
    lass = linear_model.Lasso(alpha=alphas)
    cv = cross_validate(lass, bike_feature_mtx, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Lasso Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video lasso regression
for alphas in alpha_values:
    lass = linear_model.Lasso(alpha=alphas, fit_intercept=False)
    cv = cross_validate(lass, video_feature_mtx, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Lasso Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11187376731726317, tolerance: 0.05331093559749604
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6883697346504505, tolerance: 0.0653775140172978
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09544176693637496, tolerance: 0.05740875566762194
  positive)


Lasso Regression, Alpha = 0.0001
Average Bike Training RMSE: 0.46290641561055557
Average Bike Testing/Validation RMSE: 0.4104649485720011
---------------------------------
Lasso Regression, Alpha = 0.001
Average Bike Training RMSE: 0.4622293053525649
Average Bike Testing/Validation RMSE: 0.41053018396641716
---------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.058682800617390285, tolerance: 0.05740875566762194
  positive)


Lasso Regression, Alpha = 0.01
Average Bike Training RMSE: 0.4562246154502545
Average Bike Testing/Validation RMSE: 0.41277439398717536
---------------------------------
Lasso Regression, Alpha = 0.1
Average Bike Training RMSE: 0.5295080476607967
Average Bike Testing/Validation RMSE: 0.48345670627648146
---------------------------------
Lasso Regression, Alpha = 1
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------
Lasso Regression, Alpha = 10
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------
Lasso Regression, Alpha = 100
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------
Lasso Regression, Alpha = 1000
Average Bike Training RMSE: 0.993377843147176
Average Bike Testing/Validation RMSE: 0.994645457559705
---------------------------------

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.898672356612224, tolerance: 6.1798795812682545
  positive)


Lasso Regression, Alpha = 0.0001
Average Video Training RMSE: 0.6582718551529525
Average Video Testing/Validation RMSE: 0.649339543726077
---------------------------------
Lasso Regression, Alpha = 0.001
Average Video Training RMSE: 0.6574014618438117
Average Video Testing/Validation RMSE: 0.6496469401820073
---------------------------------
Lasso Regression, Alpha = 0.01
Average Video Training RMSE: 0.6539472663850061
Average Video Testing/Validation RMSE: 0.6514170356173427
---------------------------------
Lasso Regression, Alpha = 0.1
Average Video Training RMSE: 0.6857364863145474
Average Video Testing/Validation RMSE: 0.6843970976310535
---------------------------------
Lasso Regression, Alpha = 1
Average Video Training RMSE: 0.9961400453096696
Average Video Testing/Validation RMSE: 0.9999586275305898
---------------------------------
Lasso Regression, Alpha = 10
Average Video Training RMSE: 0.9961400453096696
Average Video Testing/Validation RMSE: 0.9999586275305898
------------

In [119]:
#ridge/l2 regression


alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

#bike ridge regression
for alphas in alpha_values:
    
    ridg = linear_model.Ridge(alpha=alphas)
    cv = cross_validate(ridg, bike_feature_mtx, bike_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    # y is the target, x is the features
    bike_train_MSE = -1* cv['test_score']
    bike_test_MSE = -1* cv['train_score']

    bike_train_RMSE = np.sqrt(bike_train_MSE)
    bike_test_RMSE = np.sqrt(bike_test_MSE)

    print("Ridge Regression, Alpha = " +str(alphas))
    print("Average Bike Training RMSE: " +str(np.mean(bike_train_RMSE)))
    print("Average Bike Testing/Validation RMSE: " +str(np.mean(bike_test_RMSE)))
    print("---------------------------------")

    
#video ridge regression
for alphas in alpha_values:
    ridg = linear_model.Ridge(alpha=alphas, fit_intercept=True)
    cv = cross_validate(ridg, video_feature_mtx, video_target, return_train_score=True, scoring='neg_mean_squared_error', cv=10, verbose=False)
    video_train_MSE = -1* cv['test_score']
    video_test_MSE = -1* cv['train_score']

    video_train_RMSE = np.sqrt(video_train_MSE)
    video_test_RMSE = np.sqrt(video_test_MSE)
    print("Ridge Regression, Alpha = " +str(alphas))
    print("Average Video Training RMSE: " +str(np.mean(video_train_RMSE)))
    print("Average Video Testing/Validation RMSE: " +str(np.mean(video_test_RMSE)))
    print("---------------------------------")

Lasso Regression, Alpha = 0.0001
Average Bike Training RMSE: 0.4615732593051864
Average Bike Testing/Validation RMSE: 0.40995481876532774
---------------------------------
Lasso Regression, Alpha = 0.001
Average Bike Training RMSE: 0.4621284763941973
Average Bike Testing/Validation RMSE: 0.41003347310239774
---------------------------------
Lasso Regression, Alpha = 0.01
Average Bike Training RMSE: 0.46284797078027484
Average Bike Testing/Validation RMSE: 0.4103490244762412
---------------------------------
Lasso Regression, Alpha = 0.1
Average Bike Training RMSE: 0.4629792268297678
Average Bike Testing/Validation RMSE: 0.4104693369599798
---------------------------------
Lasso Regression, Alpha = 1
Average Bike Training RMSE: 0.46261848665318617
Average Bike Testing/Validation RMSE: 0.4104980674184879
---------------------------------
Lasso Regression, Alpha = 10
Average Bike Training RMSE: 0.4594811779519955
Average Bike Testing/Validation RMSE: 0.4108191892325195
-------------------

In [ ]:
# polynomial regression
#example: https://towardsdatascience.com/polynomial-regression-bbe8b9d97491

In [ ]:
#comment